In [1]:
!pip3 install flair

     |████████████████████████████████| 143kB 2.8MB/s 
     |████████████████████████████████| 798kB 8.5MB/s 
     |████████████████████████████████| 235kB 15.8MB/s 
     |████████████████████████████████| 983kB 16.9MB/s 
     |████████████████████████████████| 501kB 15.5MB/s 
     |████████████████████████████████| 1.0MB 20.4MB/s 
     |████████████████████████████████| 870kB 29.1MB/s 
     |████████████████████████████████| 3.7MB 40.3MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116679 sha256=b0de654fd6faf3715a791427126d49f54122fd42a378053fb2085b4326558fed
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993193 sha256=3750123bc7195de2181e014027018409fdea62843792378921907a5df8c8f30a
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
  Created wheel for sqlitedict: 

In [2]:
!git clone -l -s http://github.com/RajaHusnainGauher/FYP.git


Cloning into 'FYP'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 2888 (delta 0), reused 3 (delta 0), pack-reused 2882
Receiving objects: 100% (2888/2888), 12.86 MiB | 24.43 MiB/s, done.
Resolving deltas: 100% (473/473), done.


In [3]:
import numpy as np
import pandas as pd
from flair.embeddings import WordEmbeddings, StackedEmbeddings, BertEmbeddings
from flair.data import Sentence
from keras.layers import Input, Dense, GRU, Bidirectional, Flatten
from keras.optimizers import Adam
from keras.models import Model

Using TensorFlow backend.


In [0]:
import pandas as pd 
data = pd.read_csv("/content/FYP/data/ml/wassen.csv")

In [0]:
from sklearn.utils import shuffle
data = shuffle(data)

In [6]:
import re
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [0]:
def URLS(text):
  try:
    out=re.sub(r'@[A-Za-z0-9]+','',text)
    return re.sub('http:?//@[A-Za-z0-9./]+','',out)
  except:
    print("error")




In [0]:
def alpha(text):
  try:
    out="".join([char for char in text if char not in string.punctuation])
    return out
  except:
    print("error")

In [9]:
data['tweet']=data['tweet'].apply(lambda x:URLS(x))
data['tweet']=data['tweet'].apply(lambda x:alpha(x))

error
error


In [0]:
dataset=data

In [11]:
dataset

,Unnamed: 0,Unnamed: 0.1,index,tweet,class,norm
13781,2454,2454,16489,RT show How would poor Lynn and Tony feel losi...,none,2
14543,426,426,6939,A 1 Are you fuckin serious Fair suck of the sa...,none,2
7638,7638,7638,4340,Call me sexist or whatever it is but theres a...,sexism,1
14141,24,24,7964,Calling Kat and Andre the most despised people...,none,2
9298,9298,9298,8073,RT Cant get enough of My Kitchen Rules at the...,none,2
...,...,...,...,...,...,...
11332,5,5,12621,awwwww P,none,2
6557,6557,6557,16488,seriously though a FOURTH instant restaurant r...,none,2
13583,2256,2256,7937,Producers are certainly capitalising on the ha...,none,2
13094,1767,1767,4009,RT Call me sexist or Old fashioned but i pref...,sexism,1


In [0]:
texts = ["He played with his cat in the garden.",
         "They went for a walk with the dog.",
         "She keeps the bird in a large cage.",
         "The engine was broken so we had to repair the old car.",
         "The truck was late because of a huge traffic jam.",
         "He loved to go for a ride on his motorcycle in the late summer."]
classes = [0,0,0,1,1,1]

dataset = pd.DataFrame()
dataset["text"] = texts
dataset["class"] = classes

In [12]:
stacked_embedding = StackedEmbeddings([BertEmbeddings('bert-base-uncased')])

In [13]:
rows = dataset.shape[0]
i = 1000
sets = 0
while(i < rows):
  i = i + 1000
  sets =sets + 1
mid1 = int(sets/2)*1000
mid2 = sets*1000
dataset1 = dataset[0:mid1]
dataset2 = dataset[mid1:mid2]
print('dataset1 shape: ',dataset1.shape)
print('dataset2 shape: ',dataset2.shape)
btch_size = dataset1.shape[0]

dataset1 shape:  (8000, 6)
dataset2 shape:  (8000, 6)


In [0]:
def generateTrainingData(dataset, batch_size, max_length, num_classes, emb_size):
  
  x_batch = []
  y_batch = []
  while True:
    data = dataset.sample(frac=1)
    for index, row in data.iterrows():
        try:
          my_sent = row["tweet"]
          sentence = Sentence(my_sent)
          stacked_embedding.embed(sentence)
          
          
          x = []
          for token in sentence:
            x.append(token.embedding.cpu().detach().numpy())
            if len(x) == max_length:
              break
          
          while len(x) < max_length:
            x.append(np.zeros(emb_size))
          
          y = np.zeros(num_classes)
          y[row["norm"]] = 1    #'class' for other datasets 'norm' for wassen
          
          x_batch.append(x)            
          y_batch.append(y)
        except:
          print("An exception occurred")

        if len(y_batch) == batch_size:
          return np.array(x_batch), np.array(y_batch)

          x_batch = []
          y_batch = []

In [15]:
#Embedd Dataset1
x,y = generateTrainingData(dataset=dataset1, batch_size=btch_size, max_length=20, num_classes=3, emb_size=3072)

2020-03-10 15:00:42,480 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?


In [0]:
y

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [0]:
# del variables for union dataset to clear ram
del dataset1
del dataset
del data

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)#.20 for union dataset

In [18]:
X_train.shape
y_train.shape

(6000, 3)

In [0]:
from keras.layers import Dropout,Conv1D,MaxPool1D,GlobalMaxPooling1D

In [0]:
kernel_size = 1
filters = 1000

In [21]:
def declare_model(batch_size, max_len, emb_size, gru_size, num_classes):
 
  sample = Input(batch_shape=(batch_size, max_len, emb_size))
  dropout=Dropout(0.2)(sample)
  conv1=Conv1D(filters,kernel_size,padding='valid',activation='relu',strides=1)(dropout)
 
  mp=MaxPool1D(pool_size=4)(conv1)
  gru_out = Bidirectional(GRU(gru_size, return_sequences=True))(mp)


  gru_out = Flatten()(gru_out)
  predictions = Dense(num_classes, activation='sigmoid')(gru_out)

  model = Model(inputs=sample, outputs=[predictions])
  model.compile(optimizer=Adam(),loss='categorical_crossentropy', metrics=["acc"])
  print(model.summary())

  return model
m = declare_model(batch_size=100, max_len=20, emb_size=3072, gru_size=100, num_classes=3)




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (100, 20, 3072)           0         
_________________________________________________________________
dropout_1 (Dropout)          (100, 20, 3072)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (100, 20, 1000)           3073000   
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (100, 5, 1000)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (100, 5, 200)             660600    
_________________________________________________________________
flatten_1 (Flatten)          (100, 1000)               0         
___

In [22]:
m.fit(X_train,y_train,batch_size=100,epochs=10,validation_split=0.1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 5400 samples, validate on 600 samples
Epoch 1/10





5400/5400 [==============================] - 13s 2ms/step - loss: 0.6115 - acc: 0.6752 - val_loss: 0.5331 - val_acc: 0.6733
Epoch 2/10
5400/5400 [==============================] - 4s 815us/step - loss: 0.4944 - acc: 0.6831 - val_loss: 0.5012 - val_acc: 0.6733
Epoch 3/10
5400/5400 [==============================] - 4s 818us/step - loss: 0.4367 - acc: 0.6831 - val_loss: 0.4918 - val_acc: 0.6733
Epoch 4/10
5400/5400 [==============================] - 4s 821us/step - loss: 0.3907 - acc: 0.7130 - val_loss: 0.5009 - val_acc: 0.7800
Epoch 5/10
5400/5400 [==============================] - 4s 790us/step - loss: 0.3002 - acc: 0.8735 - val_loss: 0.4402 - val_acc: 0.8283
Epoch 6/10
5400/5400 [==============================] - 4s 783us/step - loss: 0.2063 - acc: 0.9174 - val_loss: 0.5100 - val_acc: 0.8217
Epoch 7/10
5400/5400 [==============

In [0]:
weights = m.get_weights()  # Retrieves the state of the model.
m.set_weights(weights)
del x
del y
del X_train
del y_train
#del dataset1

In [0]:
#Embedd Dataset2
x1,y1 = generateTrainingData(dataset=dataset2, batch_size=btch_size, max_length=20, num_classes=3, emb_size=3072)

In [0]:
del dataset2

In [0]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(x1, y1, test_size=0.25, random_state=42)#.20 for union dataset

In [0]:
del x1
del y1

In [29]:
m.fit(X_train1,y_train1,batch_size=100,epochs=10,validation_split=0.1)

Train on 5400 samples, validate on 600 samples
Epoch 1/10
5400/5400 [==============================] - 4s 810us/step - loss: 0.5122 - acc: 0.8080 - val_loss: 0.4435 - val_acc: 0.8117
Epoch 2/10
5400/5400 [==============================] - 4s 776us/step - loss: 0.3634 - acc: 0.8543 - val_loss: 0.4883 - val_acc: 0.8117
Epoch 3/10
5400/5400 [==============================] - 4s 767us/step - loss: 0.2821 - acc: 0.8885 - val_loss: 0.4251 - val_acc: 0.8467
Epoch 4/10
5400/5400 [==============================] - 4s 753us/step - loss: 0.1835 - acc: 0.9337 - val_loss: 0.4893 - val_acc: 0.8150
Epoch 5/10
5400/5400 [==============================] - 4s 757us/step - loss: 0.1024 - acc: 0.9656 - val_loss: 0.5168 - val_acc: 0.8350
Epoch 6/10
5400/5400 [==============================] - 4s 755us/step - loss: 0.0497 - acc: 0.9889 - val_loss: 0.5813 - val_acc: 0.8200
Epoch 7/10
5400/5400 [==============================] - 4s 755us/step - loss: 0.0254 - acc: 0.9946 - val_loss: 0.6517 - val_acc: 0.8233
E

In [0]:
del X_train1
del y_train1

In [31]:
print(X_test1.shape)
print(y_test1.shape)
print(X_test.shape)
print(y_test.shape)
X_test_complete = np.concatenate((X_test1, X_test))
print(X_test_complete.shape)

(2000, 20, 3072)
(2000, 3)
(2000, 20, 3072)
(2000, 3)
(4000, 20, 3072)


In [32]:
y_test_complete = np.concatenate((y_test1, y_test))
print(y_test_complete.shape)

(4000, 3)


In [0]:
def generatePredictionData(dataset, batch_size, max_length, num_classes, emb_size):
  
  x_batch = []
  while True:
    for text in dataset:
 
        my_sent = text
        sentence = Sentence(my_sent)
        stacked_embedding.embed(sentence)
        
        x = []
        for token in sentence:
          x.append(token.embedding.cpu().detach().numpy())
          if len(x) == max_length:
            break
        
        while len(x) < max_length:
          x.append(np.zeros(emb_size))
        

        
        x_batch.append(x)            

        if len(x_batch) == batch_size:
          yield np.array(x_batch)

          x_batch = []

In [33]:
test_rows = int(y_test_complete.shape[0]/100)
test_rows

40

In [34]:
i=0
new=[]
new=np.asarray(new)
while i<test_rows:
  new=np.concatenate((new,np.argmax(m.predict(X_test_complete[i*100:(i+1)*100], steps=1), axis=1)),axis=None)
  i+=1
print(new)

[0. 1. 2. ... 2. 2. 2.]


In [0]:
n=np.asarray(y_test_complete)

In [36]:
n.shape

(4000, 3)

In [0]:
i=0
L=[]
while(i<n.shape[0]):
  if(n[i][0]==1):
    L.append(0)
  elif(n[i][1]==1):
    L.append(1)
  else:
    L.append(2)
  i+=1

In [0]:
n1=np.asarray(L)

In [39]:
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score
print(f1_score(n1,new,average="micro"))

0.825


In [0]:
#save and load model
from keras.models import model_from_json
from keras.models import load_model

In [0]:
# serialize model to JSON
#  the keras model which is trained is defined as 'model' in this example
model_json = m.to_json()


with open("wassen_model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
m.save_weights("wassen_weights.h5")

In [0]:
# load json and create model
json_file = open('tdavidson_model.json', 'r')

loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("tdavidson_weights.h5")
print("Loaded model from disk")

loaded_model.save('model_1.hdf5')
loaded_model=load_model('tdavidson_weights.hdf5')

Loaded model from disk


/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [59]:
#here you can enter string after loading your model and test your model

num = input ("Enter a string \n") 
sentence = Sentence(num)
stacked_embedding.embed(sentence)     
x = []
i=0
for token in sentence:
  x.append(token.embedding.cpu().detach().numpy())
  

while len(x) < 20:
          x.append(np.zeros(3072))
  
#-------------------------------------------------------------------
x1=[]
i=0
while i<100:
  x1.append(x)
  i=i+1
#-------------------------------------------------------------------
x1=np.asarray(x1)
#-------------------------------------------------------------------
i=0
new=[]
new=np.asarray(new)
while i<1:
  new=np.concatenate((new,np.argmax(m.predict(x1[i*100:(i+1)*100], steps=1), axis=1)),axis=None)
  i+=1

if(new[0]==0):
  print("racist")


if(new[0]==2):
  print("None")


if(new[0]==1):
  print("Sexist")
#-------------------------------------------------------------------



Enter a string 
Ali was such a bully that i'm surprised he's still doing this
None


In [0]:
Womens don't deserve to be leaders
Muslims should go back to their countries
Ali was such a bully that i'm surprised he's still doing this